# Fair Binary Classification with SearchFair on CelebA and Adult

Here, we show how to use SearchFair on two datasets: CelebA and Adult

## Imports

We start by importing SearchFair from the installed package.

In [48]:
!pip install aif360

     |████████████████████████████████| 175 kB 5.6 MB/s eta 0:00:01
     |████████████████████████████████| 356 kB 49.9 MB/s eta 0:00:01
  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-py3-none-any.whl size=30182 sha256=005e256803c214b3cdeebcebb326e0a24e9f29081db9f2f2d86a28d4ca3308d6
  Stored in directory: /Users/designwork/Library/Caches/pip/wheels/6a/37/3e/d9e8ebaf73956a3ebd2ee41869444dbd2a702d7142bcf93c42
  Created wheel for shap: filename=shap-0.39.0-cp38-cp38-macosx_10_9_x86_64.whl size=415958 sha256=51ee4de50cb26703f63c3fdc80d5c1eeb1615831874766d3435440c362068278
  Stored in directory: /Users/designwork/Library/Caches/pip/wheels/3d/c9/06/734ed80d6d61fad331974bf62017b4ea6b33488082b9f5e67e
Successfully built memory-profiler shap


In [49]:
from aif360 import datasets

In [54]:
datasets.CompasDataset[0]

TypeError: 'ABCMeta' object is not subscriptable

In [1]:
from searchfair import SearchFair
import pandas as pd

Second, we load some necessary methods and numpy.

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np

# The optimization does not always comply with the new cvxpy dpp disciplined programming rules. 
# but this is not a problem. 
import warnings
warnings.filterwarnings('ignore')

# The CelebA dataset

On the Celebrity Faces dataset we are given descriptions of celebrity faces, with 40 binary attributes. Here, we use the Attribute 'Smiling' as the class label, and sex as the sensitive attribute. 

In [3]:
!Ls

Data Cleaning Recidivism.ipynb real_data.ipynb
__pycache__                    recividism_black_white.csv
get_real_data.py               recividism_cleaned.csv
get_synthetic_data.py          recividism_white_nonwhite.csv
new_Recidivism.csv             toy_data.ipynb
real_data - recidivism.ipynb   utils.py


In [4]:
# importing cleaned recidivism data from IOWA
# https://data.iowa.gov/Correctional-System/3-Year-Recidivism-for-Offenders-Released-from-Pris/mw8r-vqy4

In [38]:
df = pd.read_csv('new_Recidivism.csv')

In [39]:
df

,Unnamed: 0,Release Type_Discharged - Expiration of Sentence,Release Type_Discharged – End of Sentence,Release Type_Parole,Release Type_Parole Granted,Release Type_Paroled to Detainer - INS,Release Type_Paroled to Detainer - Iowa,Release Type_Paroled to Detainer - Out of State,Release Type_Paroled to Detainer - U.S. Marshall,Release Type_Paroled w/Immediate Discharge,...,Offense Subtype_Sex Offender Registry/Residency,Offense Subtype_Special Sentence Revocation,Offense Subtype_Stolen Property,Offense Subtype_Theft,Offense Subtype_Traffic,Offense Subtype_Trafficking,Offense Subtype_Vandalism,Offense Subtype_Weapons,Race,Recidivism
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-1,1
1,4,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-1,1
2,7,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-1,1
3,9,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-1,1
4,12,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12573,15575,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,-1
12574,15576,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,-1
12575,15579,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,-1
12576,15580,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,-1


In [40]:
s_data = df.Race.values

s_data.shape

y_data = df.Recidivism.values

x_data = df[df.columns[1:-2]].values

x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_data, y_data, s_data, train_size=1200, shuffle=True)

In [35]:
import get_real_data as get_data

# Load Data
# x_data, y_data, s_data = get_data.get_celebA_data(load_data_size=None)
# Train Test split. Here, we choose a small number to reduce running time.
train_size = 10000
x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_data, y_data, s_data, train_size=train_size, shuffle=True)

In [10]:
!ls

Data Cleaning Recidivism.ipynb real_data.ipynb
__pycache__                    recividism_black_white.csv
get_real_data.py               recividism_cleaned.csv
get_synthetic_data.py          recividism_white_nonwhite.csv
new_Recidivism.csv             toy_data.ipynb
real_data - recidivism.ipynb   utils.py


In [11]:
x_data.shape

(12578, 56)

In [12]:
def print_clf_stats(model, x_train, x_test, y_train, y_test, s_train, s_test):
    train_acc = ut.get_accuracy(np.sign(model.predict(x_train)), y_train)
    test_acc = ut.get_accuracy(np.sign(model.predict(x_test)), y_test)
    test_DDP, test_DEO = ut.compute_fairness_measures(model.predict(x_test), y_test, s_test)
    train_DDP, train_DEO = ut.compute_fairness_measures(model.predict(x_train), y_train, s_train)

    print(10*'-'+"Train"+10*'-')
    print("Accuracy: %0.4f%%" % (train_acc * 100))
    print("DDP: %0.4f%%" % (train_DDP * 100), "DEO: %0.4f%%" % (train_DEO * 100))
    print(10*'-'+"Test"+10*'-')
    print("Accuracy: %0.4f%%" % (test_acc * 100))
    print("DDP: %0.4f%%" % (test_DDP * 100), "DEO: %0.4f%%" % (test_DEO * 100))

In [13]:
import utils as ut
ut.print_data_stats(s_data, y_data)

Total data points: 12578
# non-protected examples: 8000
# protected examples: 4578
# non-protected examples in positive class: 2000 (25.0%)
# protected examples in positive class: 2278 (49.8%)


## Basic model, no fairness constraint

In [69]:
len(x_train)

1200

In [14]:
from sklearn import linear_model
reg = linear_model.LogisticRegression()
x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_data, y_data, s_data, train_size=train_size, shuffle=True)
reg.fit(x_train,y_train)
print_clf_stats(reg,x_train,x_test,y_train,y_test,s_train,s_test)

----------Train----------
Accuracy: 67.5900%
DDP: -5.6941% DEO: -20.5393%
----------Test----------
Accuracy: 65.5159%
DDP: -4.2844% DEO: -17.6617%


In [37]:
num_cycles = 5
train_size = 10000
avg_train_acc = []
avg_test_acc = []
avg_test_DDP = []
avg_train_DDP = []

avg_test_DEO = []
avg_train_DEO =[]
for i in range(num_cycles):
    x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_data, y_data, s_data, train_size=train_size, shuffle=True)
    model = linear_model.LogisticRegression()
    model.fit(x_train,y_train)  

    train_acc = ut.get_accuracy(np.sign(model.predict(x_train)), y_train)
    test_acc = ut.get_accuracy(np.sign(model.predict(x_test)), y_test)
    test_DDP, test_DEO = ut.compute_fairness_measures(model.predict(x_test), y_test, s_test)
    train_DDP, train_DEO = ut.compute_fairness_measures(model.predict(x_train), y_train, s_train)

    avg_train_acc.append(train_acc)
    avg_test_acc.append(test_acc)
    avg_test_DDP.append(test_DDP)
    avg_train_DDP.append(train_DDP)
    avg_test_DEO.append(test_DEO)
    avg_train_DEO.append(train_DEO)

print(avg_train_acc)
print(np.mean(avg_train_acc))
print(np.mean(avg_test_acc))

print(np.mean(avg_train_DDP))
print(np.mean(avg_train_DEO))

print('Test DDP',np.mean(avg_test_DDP))
print('Test DEO',np.mean(avg_test_DEO))


[0.668, 0.6693, 0.6739, 0.6714, 0.6669]
0.6698999999999999
0.6757951900698215
-0.052996601132817435
-0.17962525231249118
Test DDP -0.04362384121586158
Test DEO -0.17551939070146755


## Creating a SearchFair Model

### Demographic Parity

To learn a classifier with SearchFair, we need to choose a kernel between 'linear' and 'rbf', and we need to choose a fairness notion - either Demographic Parity (DDP) or Equality of Opportunity (DEO). 

In [41]:
fairness_notion = 'DEO' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'linear' # 'linear', 'rbf'
verbose = True # True = SearchFair output, 2 = show also solver progress
x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_data, y_data, s_data, train_size=1000, shuffle=True)

# Regularization Parameter beta
reg_beta = 0.0001
linear_model_DDP = SearchFair(reg_beta=reg_beta, kernel=kernel, fairness_notion=fairness_notion, verbose=verbose, stop_criterion=0.01)
linear_model_DDP.fit(x_train, y_train, s_train=s_train)
print_clf_stats(linear_model_DDP, x_train, x_test, y_train, y_test, s_train, s_test)

Preprocessing...
Testing lambda_min: 0.00
Obtained: DEO = 0.0054 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DEO = -0.4086 with lambda = 1.0000
Classifier is fair enough with lambda = 0.0000
----------Found Lambda 0.0000 with fairness 0.0054----------
----------Train----------
Accuracy: 66.5000%
DDP: -0.2046% DEO: 0.5376%
----------Test----------
Accuracy: 65.8404%
DDP: -2.0640% DEO: -4.8124%


In [42]:
fairness_notion = 'DEO' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'linear' # 'linear', 'rbf'
verbose = True # True = SearchFair output, 2 = show also solver progress
x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_data, y_data, s_data, train_size=1000, shuffle=True)

# Regularization Parameter beta
reg_beta = 0.0001
linear_model_DDP = SearchFair(reg_beta=reg_beta, kernel=kernel, fairness_notion=fairness_notion, verbose=verbose, stop_criterion=0.01)
linear_model_DDP.fit(x_train, y_train, s_train=s_train)
print_clf_stats(linear_model_DDP, x_train, x_test, y_train, y_test, s_train, s_test)

Preprocessing...
Testing lambda_min: 0.00


KeyboardInterrupt: 

In [86]:
fairness_notion = 'DDP' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'linear' # 'linear', 'rbf'
verbose = True # True = SearchFair output, 2 = show also solver progress
x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_data, y_data, s_data, train_size=1200, shuffle=True)

# Regularization Parameter beta
reg_beta = 0.0001
linear_model_DDP = SearchFair(reg_beta=reg_beta, kernel=kernel, fairness_notion=fairness_notion, verbose=verbose, stop_criterion=0.01)
linear_model_DDP.fit(x_train, y_train, s_train=s_train)
print_clf_stats(linear_model_DDP, x_train, x_test, y_train, y_test, s_train, s_test)

Preprocessing...
Testing lambda_min: 0.00
Obtained: DDP = 0.0000 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DDP = 0.0000 with lambda = 1.0000
Classifier is fair enough with lambda = 1.0000
----------Found Lambda 1.0000 with fairness 0.0000----------
----------Train----------
Accuracy: 66.1667%
DDP: 0.0000% DEO: 0.0000%
----------Test----------
Accuracy: 65.9694%
DDP: 0.0000% DEO: 0.0000%


To print out the Accuracy and the fairness notions Demographic Parity and Equality of Opportuniy, we define the following function. 

In [24]:
def print_clf_stats(model, x_train, x_test, y_train, y_test, s_train, s_test):
    train_acc = ut.get_accuracy(np.sign(model.predict(x_train)), y_train)
    test_acc = ut.get_accuracy(np.sign(model.predict(x_test)), y_test)
    test_DDP, test_DEO = ut.compute_fairness_measures(model.predict(x_test), y_test, s_test)
    train_DDP, train_DEO = ut.compute_fairness_measures(model.predict(x_train), y_train, s_train)

    print(10*'-'+"Train"+10*'-')
    print("Accuracy: %0.4f%%" % (train_acc * 100))
    print("DDP: %0.4f%%" % (train_DDP * 100), "DEO: %0.4f%%" % (train_DEO * 100))
    print(10*'-'+"Test"+10*'-')
    print("Accuracy: %0.4f%%" % (test_acc * 100))
    print("DDP: %0.4f%%" % (test_DDP * 100), "DEO: %0.4f%%" % (test_DEO * 100))

Now lets see, if we obtained a fair classifier with respect to the fairness notions we specified. 

In [30]:
print_clf_stats(linear_model_DDP, x_train, x_test, y_train, y_test, s_train, s_test)

----------Train----------
Accuracy: 64.8333%
DDP: -2.1743% DEO: -3.2929%
----------Test----------
Accuracy: 65.1637%
DDP: -0.6283% DEO: -0.7367%


In [35]:
print_clf_stats(linear_model_DDP, x_train, x_test, y_train, y_test, s_train, s_test)

----------Train----------
Accuracy: 81.8333%
DDP: 0.9718% DEO: 4.0670%
----------Test----------
Accuracy: 81.2735%
DDP: -0.8314% DEO: 1.2433%


The train DDP is small, and SearchFair succeeded to find a fair classifier. The test DDP might or might not be close to 0. This is due to the small number of points which are used to reduce running time for this example notebook. Go ahead and try it with more points!

### Equality of Opportunity

Now we try the same using a more complex rbf kernel, and we try to improve Equality of Opportunity this time. 

In [17]:
fairness_notion = 'DEO' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'rbf' # 'linear', 'rbf'
verbose = True

# Regularization Parameter beta
reg_beta = 0.0001
rbf_model_DEO = SearchFair(reg_beta=reg_beta, kernel=kernel, fairness_notion=fairness_notion, verbose=verbose)
rbf_model_DEO.fit(x_train, y_train, s_train=s_train)

# Evaluate model
print_clf_stats(rbf_model_DEO, x_train, x_test, y_train, y_test, s_train, s_test)

Preprocessing...
Testing lambda_min: 0.00
Obtained: DEO = -0.3353 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DEO = 0.4251 with lambda = 1.0000
Starting Binary Search...
----------Iteration #0----------
Testing new Lambda: 0.5000
Obtained: DEO = 0.1704 with lambda = 0.5000
----------Iteration #1----------
Testing new Lambda: 0.2500
Obtained: DEO = 0.0501 with lambda = 0.2500
----------Iteration #2----------
Testing new Lambda: 0.1250
Obtained: DEO = 0.0000 with lambda = 0.1250
Sufficient fairness obtained before maximum iterations were reached.
----------Found Lambda 0.1250 with fairness 0.0000----------
----------Train----------
Accuracy: 67.1000%
DDP: 0.0000% DEO: 0.0000%
----------Test----------
Accuracy: 65.8922%
DDP: 0.0000% DEO: 0.0000%


In [20]:
fairness_notion = 'DEO' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'rbf' # 'linear', 'rbf'
verbose = True

# Regularization Parameter beta
reg_beta = 0.0001

# training
num_cycles = 5
train_size = 1000
avg_train_acc = []
avg_test_acc = []
avg_test_DDP = []
avg_train_DDP = []

avg_test_DEO = []
avg_train_DEO =[]
for i in range(num_cycles):
    x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_data, y_data, s_data, train_size=train_size, shuffle=True)
    rbf_model_DEO = SearchFair(reg_beta=reg_beta, kernel=kernel, fairness_notion=fairness_notion, verbose=verbose)
    rbf_model_DEO.fit(x_train, y_train, s_train=s_train)
    
    rbf_model_DEO = linear_model.LogisticRegression()
    rbf_model_DEO.fit(x_train,y_train)  

    train_acc = ut.get_accuracy(np.sign(rbf_model_DEO.predict(x_train)), y_train)
    test_acc = ut.get_accuracy(np.sign(rbf_model_DEO.predict(x_test)), y_test)
    test_DDP, test_DEO = ut.compute_fairness_measures(rbf_model_DEO.predict(x_test), y_test, s_test)
    train_DDP, train_DEO = ut.compute_fairness_measures(rbf_model_DEO.predict(x_train), y_train, s_train)

    avg_train_acc.append(train_acc)
    avg_test_acc.append(test_acc)
    avg_test_DDP.append(test_DDP)
    avg_train_DDP.append(train_DDP)
    avg_test_DEO.append(test_DEO)
    avg_train_DEO.append(train_DEO)

print(avg_train_acc)
print(np.mean(avg_train_acc))
print(np.mean(avg_test_acc))

print(np.mean(avg_train_DDP))
print(np.mean(avg_train_DEO))

print('Test DDP',np.mean(avg_test_DDP))
print('Test DEO',np.mean(avg_test_DEO))

Preprocessing...
Testing lambda_min: 0.00
Obtained: DEO = 0.0000 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DEO = -0.3677 with lambda = 1.0000
Classifier is fair enough with lambda = 0.0000
----------Found Lambda 0.0000 with fairness 0.0000----------
Preprocessing...
Testing lambda_min: 0.00
Obtained: DEO = 0.0000 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DEO = -0.4641 with lambda = 1.0000
Classifier is fair enough with lambda = 0.0000
----------Found Lambda 0.0000 with fairness 0.0000----------
Preprocessing...
Testing lambda_min: 0.00
Obtained: DEO = 0.0000 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DEO = -0.3493 with lambda = 1.0000
Classifier is fair enough with lambda = 0.0000
----------Found Lambda 0.0000 with fairness 0.0000----------
Preprocessing...
Testing lambda_min: 0.00
Obtained: DEO = 0.0000 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DEO = -0.4124 with lambda = 1.0000
Classifier is fair enough with lambda = 0.0000
-----

In [36]:
fairness_notion = 'DEO' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'rbf' # 'linear', 'rbf'
verbose = True

# Regularization Parameter beta
reg_beta = 0.0001
rbf_model_DEO = SearchFair(reg_beta=reg_beta, kernel=kernel, fairness_notion=fairness_notion, verbose=verbose)
rbf_model_DEO.fit(x_train, y_train, s_train=s_train)

# Evaluate model
print_clf_stats(rbf_model_DEO, x_train, x_test, y_train, y_test, s_train, s_test)

Preprocessing...
Testing lambda_min: 0.00
Obtained: DEO = 0.2198 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DEO = -0.8758 with lambda = 1.0000
Starting Binary Search...
----------Iteration #0----------
Testing new Lambda: 0.5000
Obtained: DEO = -0.8080 with lambda = 0.5000
----------Iteration #1----------
Testing new Lambda: 0.2500
Obtained: DEO = -0.1350 with lambda = 0.2500
----------Iteration #2----------
Testing new Lambda: 0.1250
Obtained: DEO = 0.1038 with lambda = 0.1250
----------Iteration #3----------
Testing new Lambda: 0.1875
Obtained: DEO = -0.0271 with lambda = 0.1875
----------Iteration #4----------
Testing new Lambda: 0.1562
Obtained: DEO = 0.0544 with lambda = 0.1562
----------Iteration #5----------
Testing new Lambda: 0.1719
Obtained: DEO = 0.0570 with lambda = 0.1719
----------Iteration #6----------
Testing new Lambda: 0.1797
Obtained: DEO = 0.0226 with lambda = 0.1797
----------Iteration #7----------
Testing new Lambda: 0.1836
Obtained: DEO = -0.0061 wit

### Cross Validation - GridSearch

It is also possible to use GridSearchCV for the regularization paramter beta, and, if used, the width of the rbf kernel. But running this might take some time.

In [32]:
fairness_notion = 'DDP' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'rbf' # 'linear', 'rbf'
verbose = False

cv_model = SearchFair(kernel=kernel, fairness_notion=fairness_notion, verbose=verbose)

# regularization parameter beta
beta_params = [0.0001, 0.001, 0.01]
cv_params = {'reg_beta': beta_params}

if kernel == 'rbf':
    n_features = x_data.shape[1]
    default_width = 1/n_features
    order_of_magn = np.floor(np.log10(default_width))
    kernel_widths = [10**(order_of_magn), default_width, 10**(order_of_magn+1)]
    cv_params['gamma'] = kernel_widths

grid_clf = GridSearchCV(cv_model,cv_params, cv=3, verbose=1, n_jobs=1, scoring='accuracy', refit=True)
grid_clf.fit(x_train, y_train, s_train=s_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 1.0000
Classifier is fair enough with lambda = 1.0000
Classifier is fair enough with lambda = 1.0000
Classifier is fair enough with lambda = 1.0000
Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 1.0000
Classifier is fair enough with lambda = 1.0000
Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 0.0000
Classifier is fair enough with lambda = 0.0000
Classifier is

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 29.1min finished


Classifier is fair enough with lambda = 0.0000


GridSearchCV(cv=3, estimator=SearchFair(kernel='rbf'), n_jobs=1,
             param_grid={'gamma': [0.01, 0.017857142857142856, 0.1],
                         'reg_beta': [0.0001, 0.001, 0.01]},
             scoring='accuracy', verbose=1)

In [33]:
# Evaluate model
print_clf_stats(grid_clf, x_train, x_test, y_train, y_test, s_train, s_test)

----------Train----------
Accuracy: 64.0833%
DDP: 0.0000% DEO: 0.0000%
----------Test----------
Accuracy: 65.0630%
DDP: 0.0000% DEO: 0.0000%


## Adult dataset

In the fairness literature, the adult dataset is a very popular dataset. It contains US census data from 1994, where the class label indicates if the income is higher or lower than 50.000$. The binary sensitive attribute here, is the sex.

In [43]:
# Load Data
x_data, y_data, s_data = get_data.get_adult_data(load_data_size=None)
# Train Test split. Here, we choose a small number to reduce running time.
train_size = 1200
x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_data, y_data, s_data, train_size=train_size, shuffle=True)
ut.print_data_stats(s_data, y_data)

Total data points: 48842
# non-protected examples: 32650
# protected examples: 16192
# non-protected examples in positive class: 9918 (30.4%)
# protected examples in positive class: 1769 (10.9%)


In [28]:
x_train.shape

(1200, 18)

In [26]:
fairness_notion = 'DDP' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'linear' # 'linear', 'rbf'
verbose = True # True = SearchFair output, 2 = show also solver progress
# x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_data, y_data, s_data, train_size=1000, shuffle=True)

# Regularization Parameter beta
reg_beta = 0.0001
linear_model_DDP = SearchFair(reg_beta=reg_beta, kernel=kernel, fairness_notion=fairness_notion, verbose=verbose, stop_criterion=0.01)
linear_model_DDP.fit(x_train, y_train, s_train=s_train)
print_clf_stats(linear_model_DDP, x_train, x_test, y_train, y_test, s_train, s_test)

Preprocessing...
Testing lambda_min: 0.00
Obtained: DDP = 0.1942 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DDP = -0.3091 with lambda = 1.0000
Starting Binary Search...
----------Iteration #0----------
Testing new Lambda: 0.5000
Obtained: DDP = 0.0000 with lambda = 0.5000
Sufficient fairness obtained before maximum iterations were reached.
----------Found Lambda 0.5000 with fairness 0.0000----------
----------Train----------
Accuracy: 76.3333%
DDP: 0.0000% DEO: 0.0000%
----------Test----------
Accuracy: 76.0652%
DDP: 0.0000% DEO: 0.0000%


In [29]:
fairness_notion = 'DEO' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'linear' # 'linear', 'rbf'
verbose = True # True = SearchFair output, 2 = show also solver progress
# x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_data, y_data, s_data, train_size=1000, shuffle=True)

# Regularization Parameter beta
reg_beta = 0.0001
linear_model_DDP = SearchFair(reg_beta=reg_beta, kernel=kernel, fairness_notion=fairness_notion, verbose=verbose, stop_criterion=0.01)
linear_model_DDP.fit(x_train, y_train, s_train=s_train)
print_clf_stats(linear_model_DDP, x_train, x_test, y_train, y_test, s_train, s_test)

Preprocessing...
Testing lambda_min: 0.00
Obtained: DEO = 0.2066 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DEO = -0.2368 with lambda = 1.0000
Starting Binary Search...
----------Iteration #0----------
Testing new Lambda: 0.5000
Obtained: DEO = 0.0000 with lambda = 0.5000
Sufficient fairness obtained before maximum iterations were reached.
----------Found Lambda 0.5000 with fairness 0.0000----------
----------Train----------
Accuracy: 76.3333%
DDP: 0.0000% DEO: 0.0000%
----------Test----------
Accuracy: 76.0652%
DDP: 0.0000% DEO: 0.0000%


In [36]:
fairness_notion = 'DEO' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'linear' # 'linear', 'rbf'
verbose = True

# Regularization Parameter beta
reg_beta = 0.0001
rbf_model_DEO = SearchFair(reg_beta=reg_beta, kernel=kernel, fairness_notion=fairness_notion, verbose=verbose)
rbf_model_DEO.fit(x_train, y_train, s_train=s_train)

# Evaluate model
print_clf_stats(rbf_model_DEO, x_train, x_test, y_train, y_test, s_train, s_test)

Preprocessing...
Testing lambda_min: 0.00
Obtained: DEO = 0.2198 with lambda = 0.0000
Testing lambda_max: 1.00
Obtained: DEO = -0.8758 with lambda = 1.0000
Starting Binary Search...
----------Iteration #0----------
Testing new Lambda: 0.5000
Obtained: DEO = -0.8080 with lambda = 0.5000
----------Iteration #1----------
Testing new Lambda: 0.2500
Obtained: DEO = -0.1350 with lambda = 0.2500
----------Iteration #2----------
Testing new Lambda: 0.1250
Obtained: DEO = 0.1038 with lambda = 0.1250
----------Iteration #3----------
Testing new Lambda: 0.1875
Obtained: DEO = -0.0271 with lambda = 0.1875
----------Iteration #4----------
Testing new Lambda: 0.1562
Obtained: DEO = 0.0544 with lambda = 0.1562
----------Iteration #5----------
Testing new Lambda: 0.1719
Obtained: DEO = 0.0570 with lambda = 0.1719
----------Iteration #6----------
Testing new Lambda: 0.1797
Obtained: DEO = 0.0226 with lambda = 0.1797
----------Iteration #7----------
Testing new Lambda: 0.1836
Obtained: DEO = -0.0061 wit

If you want, you can also try SearchFair on Adult. 

In [44]:
num_cycles = 5
train_size = 5000
avg_train_acc = []
avg_test_acc = []
avg_test_DDP = []
avg_train_DDP = []

avg_test_DEO = []
avg_train_DEO =[]
for i in range(num_cycles):
    x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_data, y_data, s_data, train_size=train_size, shuffle=True)
    model = linear_model.LogisticRegression()
    model.fit(x_train,y_train)  

    train_acc = ut.get_accuracy(np.sign(model.predict(x_train)), y_train)
    test_acc = ut.get_accuracy(np.sign(model.predict(x_test)), y_test)
    test_DDP, test_DEO = ut.compute_fairness_measures(model.predict(x_test), y_test, s_test)
    train_DDP, train_DEO = ut.compute_fairness_measures(model.predict(x_train), y_train, s_train)

    avg_train_acc.append(train_acc)
    avg_test_acc.append(test_acc)
    avg_test_DDP.append(test_DDP)
    avg_train_DDP.append(train_DDP)
    avg_test_DEO.append(test_DEO)
    avg_train_DEO.append(train_DEO)

print(avg_train_acc)
print(np.mean(avg_train_acc))
print(np.mean(avg_test_acc))

print(np.mean(avg_train_DDP))
print(np.mean(avg_train_DEO))

print('Test DDP',np.mean(avg_test_DDP))
print('Test DEO',np.mean(avg_test_DEO))


[0.8098, 0.8156, 0.8172, 0.8202, 0.8186]
0.8162800000000001
0.816244696866019
0.21487555259867044
0.23108983327062366
Test DDP 0.22145476371909364
Test DEO 0.2498488871815907


In [ ]:
fairness_notion = 'DEO' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'rbf' # 'linear', 'rbf'
verbose = True

# Regularization Parameter beta
reg_beta = 0.0001

# training
num_cycles = 5
train_size = 1000
avg_train_acc = []
avg_test_acc = []
avg_test_DDP = []
avg_train_DDP = []

avg_test_DEO = []
avg_train_DEO =[]
for i in range(num_cycles):
    x_train, x_test, y_train, y_test, s_train, s_test = train_test_split(x_data, y_data, s_data, train_size=train_size, shuffle=True)
    rbf_model_DEO = SearchFair(reg_beta=reg_beta, kernel=kernel, fairness_notion=fairness_notion, verbose=verbose)
    rbf_model_DEO.fit(x_train, y_train, s_train=s_train)
    
    rbf_model_DEO = linear_model.LogisticRegression()
    rbf_model_DEO.fit(x_train,y_train)  

    train_acc = ut.get_accuracy(np.sign(rbf_model_DEO.predict(x_train)), y_train)
    test_acc = ut.get_accuracy(np.sign(rbf_model_DEO.predict(x_test)), y_test)
    test_DDP, test_DEO = ut.compute_fairness_measures(rbf_model_DEO.predict(x_test), y_test, s_test)
    train_DDP, train_DEO = ut.compute_fairness_measures(rbf_model_DEO.predict(x_train), y_train, s_train)

    avg_train_acc.append(train_acc)
    avg_test_acc.append(test_acc)
    avg_test_DDP.append(test_DDP)
    avg_train_DDP.append(train_DDP)
    avg_test_DEO.append(test_DEO)
    avg_train_DEO.append(train_DEO)

print(avg_train_acc)
print(np.mean(avg_train_acc))
print(np.mean(avg_test_acc))

print(np.mean(avg_train_DDP))
print(np.mean(avg_train_DEO))

print('Test DDP',np.mean(avg_test_DDP))
print('Test DEO',np.mean(avg_test_DEO))

In [23]:
fairness_notion = 'DDP' # DDP = Demographic Parity, DEO = Equality of Opportunity. 
kernel = 'rbf' # 'linear', 'rbf'
verbose = False

cv_model_adult = SearchFair(kernel=kernel, fairness_notion=fairness_notion, verbose=verbose)

# regularization parameter beta
beta_params = [0.0001, 0.001, 0.01]
cv_params = {'reg_beta': beta_params}

if kernel == 'rbf':
    n_features = x_data.shape[1]
    default_width = 1/n_features
    order_of_magn = np.floor(np.log10(default_width))
    kernel_widths = [10**(order_of_magn), default_width, 10**(order_of_magn+1)]
    cv_params['gamma'] = kernel_widths

grid_clf = GridSearchCV(cv_model_adult,cv_params, cv=3, verbose=1, n_jobs=1, scoring='accuracy')
grid_clf.fit(x_train, y_train, s_train=s_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Classifier is fair enough with lambda = 1.0000
Classifier is fair enough with lambda = 1.0000
Classifier is fair enough with lambda = 1.0000
Classifier is fair enough with lambda = 1.0000
Classifier is fair enough with lambda = 1.0000


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 74.1min finished


GridSearchCV(cv=3, estimator=SearchFair(kernel='rbf'), n_jobs=1,
             param_grid={'gamma': [0.01, 0.05555555555555555, 0.1],
                         'reg_beta': [0.0001, 0.001, 0.01]},
             scoring='accuracy', verbose=1)

In [24]:
# Evaluate model
print_clf_stats(grid_clf, x_train, x_test, y_train, y_test, s_train, s_test)

----------Train----------
Accuracy: 77.9167%
DDP: 0.0000% DEO: 0.0000%
----------Test----------
Accuracy: 76.0254%
DDP: 0.0000% DEO: 0.0000%
